# Rede Convolucional e Transfer Learning

### Trabalho 2 da disciplina Noções de Inteligência Artificial - 2/2024
### Alunos: Felipe Lopes Gibin Duarte e Matheus das Neves Fernandes

## Introdução
Neste trabalho usaremos uma rede convolucional pré-treinada (VGG) e a aplicaremos em um problema novo. Experimentaremos com a divisão da base em treinamento, validação e teste, e usaremos validação para o "early stopping" na tentativa de controlar o sobre-ajuste. A base de dados usada é a "TensorFlow Flowers Dataset". Ela contém 3670 imagens coloridas de flores pertencentes a uma de 5 classes: Margarida,
Dente-de-leão, Rosa, Girassol e Tulipa.

## 1. Preparação do Ambiente
Nesta seção, importaremos as bibliotecas necessárias e carregaremos a base de dados.

### 1.1 Importação de bibliotecas 

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

## 1.2 Carregamento dos dados

In [9]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
"tf_flowers",
split=["train[:70%]", "train[70%:]"], ## Train test split
batch_size=-1,
as_supervised=True, # Include labels
)

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

print (train_ds.shape)
print (test_ds.shape)

(2569, 150, 150, 3)
(1101, 150, 150, 3)


## 2. Treinando um MLP
Vamos comprovar que um MLP não apresenta desempenho satisfatório para o problema. Vamos evitar o overfitting ao usar a técnica de parada prematura de treinamento (early stopping), após 5 épocas sem melhora no parâmetro monitorado (neste caso a precisão de treinamento, val_accuracy), o treinamento é interrompido. Observamos que o MLP apresenta uma acurácia de cerca de 35-40%. Conforme esperado ele não resolve bem o problema de classificação.

In [15]:
model_MLP = tf.keras.Sequential([ 
    tf.keras.layers.Flatten(),  
    tf.keras.layers.Dense(256, activation='relu'),   #Camada escondida 1
    tf.keras.layers.Dense(256, activation='relu'),   #Camada escondida 2
    tf.keras.layers.Dense(5, activation='softmax')  #Camada de saída
])   

model_MLP.compile(
    optimizer=tf.keras.optimizers.Adam(),   #Otimizador Adam
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),  #Entropia cruzada
    metrics=['accuracy']
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

model_MLP.fit(
    train_ds, 
    train_labels, 
    epochs=20, 
    validation_split=0.2, 
    batch_size=32, 
    callbacks=[es]
)

Epoch 1/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 118ms/step - accuracy: 0.2432 - loss: 975.3029 - val_accuracy: 0.3405 - val_loss: 154.9264
Epoch 2/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 118ms/step - accuracy: 0.3879 - loss: 101.6346 - val_accuracy: 0.3249 - val_loss: 170.6600
Epoch 3/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - accuracy: 0.3591 - loss: 110.5299 - val_accuracy: 0.3813 - val_loss: 54.9718
Epoch 4/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 7s 114ms/step - accuracy: 0.4299 - loss: 36.6724 - val_accuracy: 0.3911 - val_loss: 27.5874
Epoch 5/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 116ms/step - accuracy: 0.4144 - loss: 25.6025 - val_accuracy: 0.3541 - val_loss: 32.2396
Epoch 6/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 118ms/step - accuracy: 0.3953 - loss: 18.1462 - val_accuracy: 0.2938 - val_loss: 20.1003
Epoch 7/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 116ms/step - accuracy: 0.3879 - loss: 12.8105 - val_accuracy: 0.3307 - val_loss: 16.6503
Epoch 8/20
65/65 ━━━━━━━━━━━━━━━━━━━━ 8s 117ms/step - accuracy: 0.3969 - loss: 11.731